# Topology Tutorial

Learn how to work with `Topology` objects in MolPy! Topology represents the connectivity graph of your molecular system - atoms, bonds, angles, and dihedrals.


## What is Topology?

`Topology` is a graph-based representation of molecular connectivity:

- **Atoms**: Vertices in the graph
- **Bonds**: Edges connecting atoms
- **Angles**: Triplets of connected atoms
- **Dihedrals**: Quadruplets of connected atoms
- **Impropers**: Special quadruplets for planarity

It's built on `igraph.Graph`, giving you powerful graph algorithms!


In [ ]:
import molpy as mp
from molpy.core.topology import Topology

## Creating a Topology

You can create an empty topology or build one from existing data:


In [ ]:
# Create an empty topology
topo = Topology()
print(f"Empty topology: {topo.n_atoms} atoms, {topo.n_bonds} bonds")

## Adding Atoms and Bonds


In [ ]:
# Add atoms
topo.add_atoms(4)  # Add 4 atoms
print(f"After adding atoms: {topo.n_atoms} atoms")

# Add bonds (connecting atoms 0-1, 1-2, 2-3)
topo.add_bond(0, 1)
topo.add_bond(1, 2)
topo.add_bond(2, 3)

print(f"After adding bonds: {topo.n_bonds} bonds")
print(f"Bonds: {topo.bonds}")

## Adding Multiple Bonds at Once


In [ ]:
# Create a new topology
topo = Topology()
topo.add_atoms(5)

# Add multiple bonds at once
bond_list = [(0, 1), (1, 2), (2, 3), (3, 4)]
topo.add_bonds(bond_list)

print(f"Topology: {topo.n_atoms} atoms, {topo.n_bonds} bonds")
print(f"Bond list: {topo.bonds}")

## Accessing Topology Properties


In [ ]:
# Build a simple chain topology
topo = Topology()
topo.add_atoms(4)
topo.add_bonds([(0, 1), (1, 2), (2, 3)])

print(f"Atoms: {topo.n_atoms}")
print(f"Bonds: {topo.n_bonds}")
print(f"Angles: {topo.n_angles}")
print(f"Dihedrals: {topo.n_dihedrals}")

# Access atom and bond arrays
print(f"Atom indices: {topo.atoms}")
print(f"Bond pairs: {topo.bonds}")
print(f"Angle triplets: {topo.angles}")
print(f"Dihedral quadruplets: {topo.dihedrals}")

## Creating Topology from Frame


In [ ]:
# Create a frame with atoms and bonds
frame = mp.Frame()
frame["atoms"] = mp.Block(
    {"x": [0.0, 1.0, 2.0, 3.0], "y": [0.0, 0.0, 0.0, 0.0], "z": [0.0, 0.0, 0.0, 0.0]}
)
frame["bonds"] = mp.Block({"i": [0, 1, 2], "j": [1, 2, 3]})

# Get topology from frame
topo = frame.get_topology()

print(f"Topology from frame: {topo.n_atoms} atoms, {topo.n_bonds} bonds")

## Creating Topology from Atomistic


In [ ]:
# Example: Get topology from Atomistic structure
from molpy.core.atomistic import Atomistic

# Create a simple chain molecule (butane-like)
atomistic = Atomistic()
c1 = atomistic.def_atom(symbol="C", xyz=[0.0, 0.0, 0.0])
c2 = atomistic.def_atom(symbol="C", xyz=[1.5, 0.0, 0.0])
c3 = atomistic.def_atom(symbol="C", xyz=[3.0, 0.0, 0.0])
c4 = atomistic.def_atom(symbol="C", xyz=[4.5, 0.0, 0.0])

atomistic.def_bond(c1, c2)
atomistic.def_bond(c2, c3)
atomistic.def_bond(c3, c4)

print(f"Created molecule: {atomistic}")

# Get topology from atomistic
topo = atomistic.get_topo()
print(f"Topology: {topo.n_atoms} atoms, {topo.n_bonds} bonds")
print(f"Angles: {topo.n_angles}, Dihedrals: {topo.n_dihedrals}")

# Generate angles and dihedrals automatically in the atomistic structure
atomistic.get_topo(gen_angle=True, gen_dihe=True)
print("\nAfter generating angles and dihedrals:")
print(f"  Angles in atomistic: {len(atomistic.angles)}")
print(f"  Dihedrals in atomistic: {len(atomistic.dihedrals)}")

## Modifying Topology


In [ ]:
# Create topology
topo = Topology()
topo.add_atoms(5)
topo.add_bonds([(0, 1), (1, 2), (2, 3), (3, 4)])

print(f"Before: {topo.n_atoms} atoms, {topo.n_bonds} bonds")

# Delete a bond
topo.delete_bond(2)  # Delete bond at index 2
print(f"After deleting bond: {topo.n_bonds} bonds")

# Delete an atom (and its bonds)
# topo.delete_atom(2)
# print(f"After deleting atom: {topo.n_atoms} atoms")

## Combining Topologies


In [ ]:
# Create two topologies
topo1 = Topology()
topo1.add_atoms(3)
topo1.add_bonds([(0, 1), (1, 2)])

topo2 = Topology()
topo2.add_atoms(3)
topo2.add_bonds([(0, 1), (1, 2)])

# Union them
topo1.union(topo2)
print(f"Combined topology: {topo1.n_atoms} atoms, {topo1.n_bonds} bonds")

## Graph Algorithms

Since Topology is based on igraph, you can use all graph algorithms:


In [ ]:
# Example: Check connectivity
topo = Topology()
topo.add_atoms(4)
topo.add_bonds([(0, 1), (1, 2), (2, 3)])

# Check if graph is connected
is_connected = topo.is_connected()
print(f"Is connected: {is_connected}")

# Get shortest path
path = topo.get_shortest_paths(0, 3)[0]
print(f"Shortest path from 0 to 3: {path}")

# Get neighbors
neighbors = topo.neighbors(1)
print(f"Neighbors of atom 1: {neighbors}")

# Get all paths between two vertices
paths = topo.get_all_simple_paths(0, 3)
print(f"All simple paths from 0 to 3: {paths}")